# Hello World! 
This is my very first machine learning project. So I expect a lot of warnings, errors and absolute-piece-of-shit validation scores. Here we go.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

img_rows, img_cols = 28, 28
num_classes = 10

def prep_data(raw):
    y = raw[:, 0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

file = "/kaggle/input/digit-recognizer/train.csv"
data = np.loadtxt(file, skiprows=1, delimiter=',')
x, y = prep_data(data)

/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv


# Building The Model

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

model = Sequential()
model.add(Conv2D(12, kernel_size=(3,3), activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(12, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(12, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(12, kernel_size=(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compiling

In [3]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fitting

In [4]:
model.fit(x, y, batch_size=100, epochs=25, validation_split=0.2)

Epoch 1/25
336/336 [==============================] - 2s 5ms/step - loss: 0.2434 - accuracy: 0.9288 - val_loss: 0.0794 - val_accuracy: 0.9758
Epoch 2/25
336/336 [==============================] - 1s 4ms/step - loss: 0.0700 - accuracy: 0.9790 - val_loss: 0.0773 - val_accuracy: 0.9764
Epoch 3/25
336/336 [==============================] - 1s 4ms/step - loss: 0.0452 - accuracy: 0.9862 - val_loss: 0.0702 - val_accuracy: 0.9790
Epoch 4/25
336/336 [==============================] - 1s 4ms/step - loss: 0.0311 - accuracy: 0.9896 - val_loss: 0.0547 - val_accuracy: 0.9840
Epoch 5/25
336/336 [==============================] - 1s 4ms/step - loss: 0.0235 - accuracy: 0.9921 - val_loss: 0.0571 - val_accuracy: 0.9845
Epoch 6/25
336/336 [==============================] - 1s 4ms/step - loss: 0.0165 - accuracy: 0.9947 - val_loss: 0.0609 - val_accuracy: 0.9823
Epoch 7/25
336/336 [==============================] - 2s 5ms/step - loss: 0.0132 - accuracy: 0.9954 - val_loss: 0.0786 - val_accuracy: 0.9789
Epoch 

# Validation Score

In [5]:
score = model.evaluate(x, y, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.023630134761333466 / Test accuracy: 0.9953333139419556


# Preparing Test Data

In [6]:
def prep_test_data(raw):
    x = raw[:,0:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x

val_file = "/kaggle/input/digit-recognizer/test.csv"
val_data = np.loadtxt(val_file, skiprows=1, delimiter=',')
x_test = prep_test_data(val_data)

# Predicting & Saving

In [7]:
predictions = model.predict_classes(x_test)

indexes = [i for i in range(1,len(val_data)+1)]
output = pd.DataFrame({'ImageId': indexes,'Label': predictions})
output.to_csv('submission.csv', index=False)